# Delta Lake Constraints

In [1]:
import pyspark
from delta import *

In [2]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

In [3]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/matthew.powers/opt/miniconda3/envs/pyspark-340-delta-240/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matthew.powers/.ivy2/cache
The jars for the packages stored in: /Users/matthew.powers/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-689eb51c-0117-4058-9228-0a87952ccc7f;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 96ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     

## Create Delta table & add check constraint

In [4]:
df = spark.createDataFrame(
    [
        (1, "foo"),
        (2, "bar"),
    ],
    ["age", "word"],
)

In [5]:
df.write.format("delta").saveAsTable("random1")

23/11/28 16:36:07 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
spark.sql("SHOW TBLPROPERTIES random1").show(truncate=False)

+----------------------+-----+
|key                   |value|
+----------------------+-----+
|delta.minReaderVersion|1    |
|delta.minWriterVersion|2    |
+----------------------+-----+



In [7]:
check = "ALTER TABLE default.random1 ADD CONSTRAINT ageIsPositive CHECK (age >= 0)"
spark.sql(check)

DataFrame[]

In [8]:
spark.sql("SHOW TBLPROPERTIES random1").show(truncate=False)

+-------------------------------+--------+
|key                            |value   |
+-------------------------------+--------+
|delta.constraints.ageispositive|age >= 0|
|delta.minReaderVersion         |1       |
|delta.minWriterVersion         |3       |
+-------------------------------+--------+



In [9]:
df = spark.createDataFrame(
    [
        (-3, "red"),
        (4, "blue"),
    ],
    ["age", "word"],
)

In [10]:
df.write.format("delta").mode("append").saveAsTable("random1")

23/11/28 16:37:15 ERROR Utils: Aborting task
org.apache.spark.sql.delta.schema.DeltaInvariantViolationException: CHECK constraint ageispositive (age >= 0) violated by row with values:
 - age : -3
	at org.apache.spark.sql.delta.schema.DeltaInvariantViolationException$.apply(InvariantViolationException.scala:60)
	at org.apache.spark.sql.delta.schema.DeltaInvariantViolationException$.apply(InvariantViolationException.scala:70)
	at org.apache.spark.sql.delta.schema.DeltaInvariantViolationException.apply(InvariantViolationException.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.delta.constraints.DeltaInvariantCheckerExec.$anonfun$doExecute$3(DeltaInvariantCheckerExec.scala:80)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.sql.execution.datasources.FileFormatDataWriter.writeWithIterator(FileFormatDataWriter.scala:92)
	at org.apache.spark.sql.execution.datasource

Py4JJavaError: An error occurred while calling o85.saveAsTable.
: org.apache.spark.sql.delta.schema.DeltaInvariantViolationException: CHECK constraint ageispositive (age >= 0) violated by row with values:
 - age : -3
	at org.apache.spark.sql.delta.schema.DeltaInvariantViolationException$.apply(InvariantViolationException.scala:60)
	at org.apache.spark.sql.delta.schema.DeltaInvariantViolationException$.apply(InvariantViolationException.scala:70)
	at org.apache.spark.sql.delta.schema.DeltaInvariantViolationException.apply(InvariantViolationException.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.delta.constraints.DeltaInvariantCheckerExec.$anonfun$doExecute$3(DeltaInvariantCheckerExec.scala:80)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.sql.execution.datasources.FileFormatDataWriter.writeWithIterator(FileFormatDataWriter.scala:92)
	at org.apache.spark.sql

In [11]:
spark.table("random1").show()

+---+----+
|age|word|
+---+----+
|  1| foo|
|  2| bar|
+---+----+



In [ ]:
import pyspark.sql.functions as F

In [ ]:
df.filter(F.col("age") >= 0).show()

In [ ]:
df.filter(F.col("age") >= 0).write.format("delta").mode("append").saveAsTable("random1")

In [ ]:
spark.table("random1").show()

## Delta Lake NOT NULL Constraint

In [ ]:
createTable = """
CREATE TABLE default.letters1 (
    letter1 STRING,
    letter2 STRING NOT NULL
  ) USING DELTA;
"""

In [ ]:
spark.sql(createTable)

In [ ]:
df = spark.createDataFrame(
    [
        ("a", "aa"),
        ("b", "bb"),
    ],
    ["letter1", "letter2"],
)

In [ ]:
df.write.format("delta").mode("append").saveAsTable("letters1")

In [ ]:
spark.table("letters1").show()

In [ ]:
spark.table("letters1").printSchema()

In [ ]:
spark.sql("SHOW TBLPROPERTIES letters1").show(truncate=False)

In [ ]:
df = spark.createDataFrame(
    [
        ("c", None),
        ("d", "dd"),
    ],
    ["letter1", "letter2"],
)

In [ ]:
df.write.format("delta").mode("append").saveAsTable("letters1")

In [ ]:
spark.table("letters1").show()

## NOT NULL Constraint vs nullable property

In [ ]:
from pyspark.sql.types import StringType, StructType

In [ ]:
schema = StructType().add("col1", StringType(), True).add("col2", StringType(), False)

df = (
    spark.read.format("csv")
    .option("header", True)
    .schema(schema)
    .load("../../data/small_file.csv")
)

In [ ]:
df.show()

In [ ]:
df.printSchema()

## Adding constraints to existing Delta tables

In [ ]:
df = spark.createDataFrame(
    [
        (-45, "hi"),
        (2, "bye"),
    ],
    ["age", "word"],
)

In [ ]:
df.write.format("delta").saveAsTable("random2")

In [ ]:
spark.table("random2").show()

In [ ]:
check = "ALTER TABLE default.random2 ADD CONSTRAINT ageIsPositive CHECK (age >= 0)"
spark.sql(check)

In [ ]:
spark.sql("SHOW TBLPROPERTIES random2").show(truncate=False)